In [1]:
import math
import torch

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

In [2]:
# use regular spaced points on the interval [0, 1]
train_X = torch.linspace(0, 1, 15, dtype=dtype, device=device)
# training data needs to be explicitly multi-dimensional
train_X = train_X.unsqueeze(1)

# sample observed values and add some synthetic noise
train_Y = torch.sin(train_X * (2 * math.pi)) + 0.15 * torch.randn_like(train_X)

In [3]:
from botorch.models import SingleTaskGP
from gpytorch.constraints import GreaterThan


model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))

In [4]:
from gpytorch.mlls import ExactMarginalLogLikelihood

mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
# set mll and all submodules to the specified dtype and device
mll = mll.to(train_X)

In [5]:
from torch.optim import SGD

optimizer = SGD([{'params': model.parameters()}], lr=0.1)

In [6]:
NUM_EPOCHS = 150

model.train()

for epoch in range(NUM_EPOCHS):
    # clear gradients
    optimizer.zero_grad()
    # forward pass through the model to obtain the output MultivariateNormal
    output = model(train_X)
    # Compute negative marginal log likelihood
    loss = - mll(output, model.train_targets)
    # back prop gradients
    loss.backward()
    # print every 10 iterations
    if (epoch + 1) % 10 == 0:
        print(
            f"Epoch {epoch+1:>3}/{NUM_EPOCHS} - Loss: {loss.item():>4.3f} "
            f"lengthscale: {model.covar_module.base_kernel.lengthscale.item():>4.3f} " 
            f"noise: {model.likelihood.noise.item():>4.3f}" 
         )
    optimizer.step()

Epoch  10/150 - Loss: 1.935 lengthscale: 0.647 noise: 1.996
Epoch  20/150 - Loss: 1.895 lengthscale: 0.603 noise: 1.848
Epoch  30/150 - Loss: 1.855 lengthscale: 0.566 noise: 1.698
Epoch  40/150 - Loss: 1.813 lengthscale: 0.534 noise: 1.546
Epoch  50/150 - Loss: 1.768 lengthscale: 0.507 noise: 1.393
Epoch  60/150 - Loss: 1.721 lengthscale: 0.484 noise: 1.241
Epoch  70/150 - Loss: 1.670 lengthscale: 0.463 noise: 1.092
Epoch  80/150 - Loss: 1.616 lengthscale: 0.445 noise: 0.946
Epoch  90/150 - Loss: 1.557 lengthscale: 0.428 noise: 0.808
Epoch 100/150 - Loss: 1.494 lengthscale: 0.413 noise: 0.679
Epoch 110/150 - Loss: 1.427 lengthscale: 0.399 noise: 0.562
Epoch 120/150 - Loss: 1.356 lengthscale: 0.385 noise: 0.457
Epoch 130/150 - Loss: 1.281 lengthscale: 0.372 noise: 0.367
Epoch 140/150 - Loss: 1.204 lengthscale: 0.361 noise: 0.290
Epoch 150/150 - Loss: 1.127 lengthscale: 0.352 noise: 0.227
